In [2]:
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt
from typing import Optional

In [4]:
import numpy as np
from scipy.stats import qmc
from typing import List, Union, Any, Dict

def generate_mixed_lhs_samples(
    n_samples: int,
    cat_list: List[Any],
    cont_range: List[float]
) -> List[Dict[str, Any]]:
    """
    產生混合維度的 LHS 抽樣：一個離散 List，一個連續區間。

    Args:
        n_samples (int): 抽樣個數。
        cat_list (List[Any]): 維度 X 的選項清單 (例如: ['Adam', 'SGD', 'RMSprop'])。
        cont_range (List[float]): 維度 Y 的連續區間 [min, max] (例如: [1e-4, 1e-2])。

    Returns:
        List[Dict[str, Any]]: 包含抽樣結果的字典列表。
    """
    # 1. 初始化 LHS 採樣器 (2個維度)
    sampler = qmc.LatinHypercube(d=2)
    raw_samples = sampler.random(n=n_samples)  # 產生 [0, 1) 之間的隨機數

    results = []
    
    # 2. 映射維度 X (離散 List)
    # 將 [0, 1] 區間等分成 len(cat_list) 份，看採樣點落在第幾個區間
    num_categories = len(cat_list)
    
    # 3. 映射維度 Y (連續區間)
    y_min, y_max = cont_range

    for i in range(n_samples):
        # 處理維度 X: 使用索引映射
        # raw_samples[i, 0] 是 0~1 的數，乘以數量後無條件捨去得到 Index
        idx = int(raw_samples[i, 0] * num_categories)
        # 確保邊界值不會溢出 (例如剛好是 1.0 的情況)
        idx = min(idx, num_categories - 1)
        val_x = cat_list[idx]
        
        # 處理維度 Y: 線性插值映射
        val_y = raw_samples[i, 1] * (y_max - y_min) + y_min
        
        results.append({
            "dimension_x": val_x,
            "dimension_y": float(val_y)
        })
        
    return results

# --- 實際測試 ---
# 假設 X 是優化器選擇，Y 是學習率 (Learning Rate)
x_options = ["Adam", "SGD", "RMSprop", "Adagrad"]
y_interval = [0.0001, 0.01]

samples = generate_mixed_lhs_samples(n_samples=10, cat_list=x_options, cont_range=y_interval)

# 打印結果
print(f"{'維度 X (類別)':<15} | {'維度 Y (連續)':<15}")
print("-" * 35)
for s in samples:
    print(f"{s['dimension_x']:<15} | {s['dimension_y']:<15.6f}")

維度 X (類別)       | 維度 Y (連續)      
-----------------------------------
RMSprop         | 0.000293       
SGD             | 0.007992       
SGD             | 0.002960       
Adam            | 0.004756       
RMSprop         | 0.003372       
Adagrad         | 0.009019       
Adam            | 0.008423       
RMSprop         | 0.006096       
Adam            | 0.001326       
Adagrad         | 0.005524       
